# Pokemon Pokedex ETL & Cleanup Script

In [14]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text  # Import text function

In [15]:
# Define connection string for SQLAlchemy
connection_string = 'mssql+pyodbc://DESKTOP-HK51R00/PokemonPokedex?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'

# Create an engine
engine = create_engine(connection_string)

# Query to pull the data
query = 'SELECT * FROM pokedex_raw'

# Pull the tables using SQLAlchemy engine
tblPokedex_raw = pd.read_sql(query, engine)

# pulling the other table data
# Classification Table
query = 'SELECT * FROM Classification'
tblClassification = pd.read_sql(query, engine)
# Ability Table
query = 'SELECT * FROM Ability'
tblAbilities = pd.read_sql(query, engine)
# GameOfOrigin Table
query = 'SELECT * FROM GameOfOrigin'
tblGameOfOrigin = pd.read_sql(query, engine)
# Types Table
query = 'SELECT * FROM Types'
tblTypes = pd.read_sql(query, engine)
#EggGroup Table
query = 'SELECT * FROM EggGroup'
tblEggGroup = pd.read_sql(query, engine)

# Dispose of the engine after use
engine.dispose()

In [16]:
pokedex_df = tblPokedex_raw.copy()
fieldsToMerge = [
  # Replace all Primary, Secondary, Hidden and Special abilities with their appropriate foreign keys
  {
  "LeftMergeField" : "PrimaryAbility",
  "RightMergeField" : "AbilityName",
  "MergingTable": tblAbilities,
  "RenameFrom" : "AbilityId",
  "RenameTo" : "PrimaryAbilityId",
  "DropsList" : ['PrimaryAbility','PrimaryAbilityDescription', 'AbilityName','AbilityDescription']
  },
  {
  "LeftMergeField" : "SecondaryAbility",
  "RightMergeField" : "AbilityName",
  "MergingTable": tblAbilities,
  "RenameFrom" : "AbilityId",
  "RenameTo" : "SecondaryAbilityId",
  "DropsList" : ['SecondaryAbility','SecondaryAbilityDescription', 'AbilityName','AbilityDescription']
  },
  {
  "LeftMergeField" : "HiddenAbility",
  "RightMergeField" : "AbilityName",
  "MergingTable": tblAbilities,
  "RenameFrom" : "AbilityId",
  "RenameTo" : "HiddenAbilityId",
  "DropsList" : ['HiddenAbility','HiddenAbilityDescription', 'AbilityName','AbilityDescription']
  },
  {
  "LeftMergeField" : "SpecialEventAbility",
  "RightMergeField" : "AbilityName",
  "MergingTable": tblAbilities,
  "RenameFrom" : "AbilityId",
  "RenameTo" : "SpecialEventAbilityId",
  "DropsList" : ['SpecialEventAbility','SpecialEventAbilityDescription', 'AbilityName','AbilityDescription']
  },
  # Replace Primary and Secondary Types with their appropriate foreign keys
  {
  "LeftMergeField" : "PrimaryType",
  "RightMergeField" : "TypeName",
  "MergingTable": tblTypes,
  "RenameFrom" : "TypeId",
  "RenameTo" : "PrimaryTypeId",
  "DropsList" : ['PrimaryType', 'TypeName']
  },
  {
  "LeftMergeField" : "SecondaryType",
  "RightMergeField" : "TypeName",
  "MergingTable": tblTypes,
  "RenameFrom" : "TypeId",
  "RenameTo" : "SecondaryTypeId",
  "DropsList" : ['SecondaryType', 'TypeName']
  },
  # Replace Classification Type with their appropriate foreign keys
  {
  "LeftMergeField" : "Classification",
  "RightMergeField" : "ClassificationName",
  "MergingTable": tblClassification,
  "RenameFrom" : "ClassificationName",
  "RenameTo" : "ClassificationName",
  "DropsList" : ['Classification', 'ClassificationName']
  },
  # Replace GameOfOrigin with their appropriate foreign keys
  {
  "LeftMergeField" : "GameofOrigin",
  "RightMergeField" : "GameName",
  "MergingTable": tblGameOfOrigin,
  "RenameFrom" : "GameId",
  "RenameTo" : "GameOfOriginId",
  "DropsList" : ['GameofOrigin', 'GameName']
  },
  # Replace Primary and Secondary EggGroups with their appropriate foreign keys
  {
  "LeftMergeField" : "PrimaryEggGroup",
  "RightMergeField" : "EggGroupType",
  "MergingTable": tblEggGroup,
  "RenameFrom" : "EggGroupId",
  "RenameTo" : "PrimaryEggGroupId",
  "DropsList" : ['PrimaryEggGroup', 'EggGroupType']
  },
  {
  "LeftMergeField" : "SecondaryEggGroup",
  "RightMergeField" : "EggGroupType",
  "MergingTable": tblEggGroup,
  "RenameFrom" : "EggGroupId",
  "RenameTo" : "SecondaryEggGroupId",
  "DropsList" : ['SecondaryEggGroup', 'EggGroupType']
  }
]

In [17]:
pokedex_df

,PokeDatabaseId,PokemonId,PokedexNumber,PokemonName,Classification,AlternateFormName,OriginalPokemonID,LegendaryType,PokemonHeight,PokemonWeight,...,SpeedEV,EVYieldTotal,CatchRate,ExperienceGrowth,ExperienceGrowthTotal,PrimaryEggGroup,SecondaryEggGroup,EggCycleCount,PreEvolutionPokemonId,EvolutionDetails
0,1,1,1,Bulbasaur,Seed Pokémon,None,NaN,None,0.7,6.9,...,0,1,45,Medium Slow,1059860,Monster,Grass,20,NaN,None
1,2,2,2,Ivysaur,Seed Pokémon,None,NaN,None,1.0,13.0,...,0,2,45,Medium Slow,1059860,Monster,Grass,20,1.0,Level 16
2,3,3,3,Venusaur,Seed Pokémon,None,NaN,None,2.0,100.0,...,0,3,45,Medium Slow,1059860,Monster,Grass,20,2.0,Level 32
3,4,4,3,Venusaur,Seed Pokémon,Mega,3.0,None,2.4,155.5,...,0,3,45,Medium Slow,1059860,Monster,Grass,20,NaN,None
4,5,1526,3,Venusaur,Seed Pokémon,Gigantamax,3.0,None,24.0,0.0,...,0,3,45,Medium Slow,1059860,Monster,Grass,20,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,1378,1873,1023,Iron Crown,Paradox Pokémon,None,NaN,None,1.6,156.0,...,0,3,10,Slow,1250000,No Eggs Discovered,None,50,NaN,None
1378,1379,1882,1024,Terapagos,Tera Pokémon,Stellar,1769.0,Legendary,1.7,77.0,...,0,3,255,Slow,1250000,No Eggs Discovered,None,0,NaN,None
1379,1380,1769,1024,Terapagos,Tera Pokémon,None,NaN,Legendary,0.2,6.5,...,0,1,255,Slow,1250000,No Eggs Discovered,None,0,NaN,None
1380,1381,1770,1024,Terapagos,Tera Pokémon,Terastal,1769.0,Legendary,0.3,16.0,...,0,4,255,Slow,1250000,No Eggs Discovered,None,0,NaN,None


In [18]:
for field in fieldsToMerge:
  pokedex_df = pd.merge(pokedex_df, field["MergingTable"], left_on=field["LeftMergeField"], right_on=field["RightMergeField"], how='left', suffixes=("_left", "_right"))
  pokedex_df.rename(columns={field["RenameFrom"]: field["RenameTo"]}, inplace=True)
  pokedex_df.drop(columns=field["DropsList"], inplace=True)

pokedex_df

,PokeDatabaseId,PokemonId,PokedexNumber,PokemonName,AlternateFormName,OriginalPokemonID,LegendaryType,PokemonHeight,PokemonWeight,MaleRatio,...,PrimaryAbilityId,SecondaryAbilityId,HiddenAbilityId,SpecialEventAbilityId,PrimaryTypeId,SecondaryTypeId,ClassificationId,GameOfOriginId,PrimaryEggGroupId,SecondaryEggGroupId
0,1,1,1,Bulbasaur,None,NaN,None,0.7,6.9,87.5,...,160,NaN,27.0,NaN,4,7.0,554,10,11,8.0
1,2,2,2,Ivysaur,None,NaN,None,1.0,13.0,87.5,...,160,NaN,27.0,NaN,4,7.0,554,10,11,8.0
2,3,3,3,Venusaur,None,NaN,None,2.0,100.0,87.5,...,160,NaN,27.0,NaN,4,7.0,554,10,11,8.0
3,4,4,3,Venusaur,Mega,3.0,None,2.4,155.5,87.5,...,271,NaN,NaN,NaN,4,7.0,554,9,11,8.0
4,5,1526,3,Venusaur,Gigantamax,3.0,None,24.0,0.0,87.5,...,160,NaN,27.0,NaN,4,7.0,554,5,11,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,1378,1873,1023,Iron Crown,None,NaN,None,1.6,156.0,0.0,...,187,NaN,NaN,NaN,11,9.0,447,7,12,NaN
1378,1379,1882,1024,Terapagos,Stellar,1769.0,Legendary,1.7,77.0,0.0,...,268,NaN,NaN,NaN,5,NaN,642,13,12,NaN
1379,1380,1769,1024,Terapagos,None,NaN,Legendary,0.2,6.5,0.0,...,267,NaN,NaN,NaN,5,NaN,642,13,12,NaN
1380,1381,1770,1024,Terapagos,Terastal,1769.0,Legendary,0.3,16.0,0.0,...,266,NaN,NaN,NaN,5,NaN,642,7,12,NaN


In [ ]:
# Deletes all rows from Pokemon table but keep the structure
with engine.begin() as conn:
    conn.execute(text("DELETE FROM Pokemon"))

In [20]:
# Rename the columns to match the database schema
pokedex_df.rename(columns={
    'PokedexNumber': 'PokedexId'
}, inplace=True)

In [21]:
pokedex_df.to_sql("Pokemon", engine, if_exists='append', index=False)
print("Table replaced successfully!")

Table replaced successfully!


In [ ]:
# Optionally export the Tables to csv files for displaying in Tableau
# Note that in practice these tables would remain in sql server with no exports, however Tableau Public does not support direct connections to SQL Server
tblAbilities.to_csv('./Table Exports/abilities.csv', index=False)
tblClassification.to_csv('./Table Exports/classification.csv', index=False)
tblGameOfOrigin.to_csv('./Table Exports/gameoforigin.csv', index=False)
tblTypes.to_csv('./Table Exports/types.csv', index=False)
tblEggGroup.to_csv('./Table Exports/egggroup.csv', index=False)
# Export the final table to a CSV file
# Pull the table from Sql Server to preserve type consistency without coeersion
query = 'SELECT * FROM Pokemon'

# Pull the tables using SQLAlchemy engine
tblPokemon = pd.read_sql(query, engine)
# Close the connection
engine.dispose()
# Export the data
tblPokemon.to_csv('./Table Exports/Pokemon_Pokedex.csv', index=False)